In [ ]:
import numpy as np
import cvxpy as cp
import time
import collections

def solve_closed_form(b, c, groups, Q, alpha, beta):
    """
    Calculates the optimal allocation d using the derived closed-form solution.
    
    Args:
        b (np.array): Vector of benefit coefficients.
        c (np.array): Vector of cost coefficients.
        groups (dict): Dictionary mapping group index k to a list of individual indices i.
        Q (float): Total budget.
        alpha (float): Outer fairness parameter.
        beta (float): Inner fairness parameter.

    Returns:
        np.array: The optimal allocation vector d*.
    """
    print("--- Calculating solution via Closed-Form Formula ---")
    start_time = time.time()

    if beta == 1:
        raise ValueError("The closed-form solution is not defined for beta = 1.")
    
    n = len(b)
    d_star = np.zeros(n)
    
    # Calculate the composite exponent gamma
    gamma = beta + alpha - alpha * beta
    if gamma == 0:
        raise ValueError("gamma (alpha + beta - alpha*beta) cannot be zero.")

    # Calculate group-specific terms S_k and H_k
    S = {}
    H = {}
    for k, indices in groups.items():
        b_k = b[indices]
        c_k = c[indices]
        
        # S_k = sum(c_j^-(1-beta)/beta * b_j^(1-beta)/beta)
        S[k] = np.sum(
            (c_k ** (-(1 - beta) / beta)) * (b_k ** ((1 - beta) / beta))
        )
        # H_k = sum(c_j^(beta-1)/beta * b_j^(1-beta)/beta)
        H[k] = np.sum(
            (c_k ** ((beta - 1) / beta)) * (b_k ** ((1 - beta) / beta))
        )

    # Calculate the global normalization constant Sigma
    sigma_sum = 0
    for k, indices in groups.items():
        G_k = len(indices)
        term = (
            H[k] *
            (G_k ** ((alpha - 1) / gamma)) *
            (S[k] ** (-alpha / gamma)) *
            ((1 - beta) ** (alpha / gamma))
        )
        sigma_sum += term
        
    if sigma_sum == 0:
        raise ValueError("Normalization constant Sigma is zero. Cannot divide by zero.")

    # Assemble the final solution for each d_i
    for k, indices in groups.items():
        G_k = len(indices)
        
        # Group-specific part of the formula
        group_factor = (
            (G_k ** ((alpha - 1) / gamma)) *
            (S[k] ** (-alpha / gamma)) *
            ((1 - beta) ** (alpha / gamma))
        )
        
        for i in indices:
            individual_factor = (c[i] ** (-1 / beta)) * (b[i] ** ((1 - beta) / beta))
            d_star[i] = (Q / sigma_sum) * group_factor * individual_factor

    end_time = time.time()
    print(f"Calculation completed in {end_time - start_time:.6f} seconds.")
    return d_star


def solve_with_cvxpy(b, c, groups, Q, alpha, beta, solver=cp.MOSEK):
    """
    Solves the coupled alpha-fairness problem using CVXPY.

    This function uses a reformulation with slack variables to ensure the problem
    is DCP (Disciplined Convex Programming) compliant.
    """
    print(f"\n--- Solving problem with CVXPY and {solver} ---")
    start_time = time.time()

    n = len(b)
    k_count = len(groups)
    
    # Decision variables
    d = cp.Variable(n, name='d', nonneg=True)
    
    # Slack variables for the reformulation u_k <= mu_k(d)
    u = cp.Variable(k_count, name='u',nonneg=True)

    # Constraints
    constraints = [cp.sum(cp.multiply(c, d)) <= Q]
    
    group_utilities = []
    for k, indices in groups.items():
        G_k = len(indices)
        
        # Slice the variables and parameters for the current group
        d_k = d[indices]
        b_k = b[indices]

        # Calculate g_beta terms for each individual in the group
        if beta == 1:
            # Handle log utility case
            g_terms = cp.log(cp.multiply(b_k, d_k))
        else:
            # The power utility function g_beta
            g_terms = cp.power(cp.multiply(b_k, d_k), 1 - beta) / (1-beta)
        
        # The average group utility mu_k
        mu_k = (1 / G_k) * (cp.sum(g_terms))
        
        # Reformulation constraint: u_k <= mu_k. CVXPY will correctly handle the
        # curvature of g_beta (concave) for this inequality.
        constraints.append(u[k] <= mu_k)

    # Objective Function
    if alpha == 1:
        # Logarithmic utility for f_alpha
        obj_terms = cp.sum(cp.log(u))
    else:
        # Power utility for f_alpha. CVXPY correctly identifies this as concave
        # for any alpha > 0, alpha != 1.
        obj_terms = cp.sum(cp.power(u, 1 - alpha) / (1 - alpha))

    objective = cp.Maximize(obj_terms)
    
    # Define and solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=solver, verbose=True)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        raise RuntimeError(f"CVXPY solver failed with status: {problem.status}")

    end_time = time.time()
    print(f"Solver finished in {end_time - start_time:.6f} seconds.")
    return d.value


if __name__ == '__main__':
    # --- 1. Define Problem Parameters ---
    N_INDIVIDUALS = 50
    N_GROUPS = 5
    ALPHA = 1.5  # Try values like 0.5, 2.0, etc. (but not 1.0)
    BETA = 2.5   # Try values like 0.5, 2.0, etc. (but not 1.0)
    Q_BUDGET = 1000.0



    print("="*60)
    print("Coupled Alpha-Fairness Solution Verification")
    print(f"Parameters: N={N_INDIVIDUALS}, K={N_GROUPS}, alpha={ALPHA}, beta={BETA}, Q={Q_BUDGET}")
    print("="*60)

    # --- 2. Generate Random Problem Data ---
    np.random.seed(42)
    # Generate random positive benefit and cost coefficients
    b_coeffs = np.random.uniform(1, 10, size=N_INDIVIDUALS)
    c_coeffs = np.random.uniform(0.5, 5, size=N_INDIVIDUALS)

    # Assign individuals to groups randomly
    group_assignments = np.random.randint(0, N_GROUPS, size=N_INDIVIDUALS)
    groups_dict = collections.defaultdict(list)
    for i, k in enumerate(group_assignments):
        groups_dict[k].append(i)
    # Ensure all groups are present
    while len(groups_dict) < N_GROUPS:
         groups_dict[N_GROUPS - 1].append(groups_dict[0].pop())


    # --- 3. Run Both Methods ---
    try:
        # Calculate d* using the derived analytical formula
        d_closed = solve_closed_form(b_coeffs, c_coeffs, groups_dict, Q_BUDGET, ALPHA, BETA)
        
        # Calculate d* using the CVXPY convex solver
        d_cvxpy = solve_with_cvxpy(b_coeffs, c_coeffs, groups_dict, Q_BUDGET, ALPHA, BETA)

        # --- 4. Compare the Results ---
        print("\n--- Comparison of Results ---")
        print("First 5 elements of d* from Closed-Form:", d_closed[:5])
        print("First 5 elements of d* from CVXPY Solver:", d_cvxpy[:5])

        # Calculate the difference
        difference = np.linalg.norm(d_closed - d_cvxpy)
        print(f"\nEuclidean norm of the difference between the two solution vectors: {difference:.8f}")

        if np.allclose(d_closed, d_cvxpy, rtol=1e-3):
            print("\n✅ SUCCESS: The closed-form solution matches the CVXPY solver's result.")
        else:
            print("\n❌ FAILURE: The solutions do not match.")
            
        # Verify budget usage
        budget_closed = np.sum(c_coeffs * d_closed)
        budget_cvxpy = np.sum(c_coeffs * d_cvxpy)
        print(f"\nBudget used (Closed-Form): {budget_closed:.4f} / {Q_BUDGET}")
        print(f"Budget used (CVXPY):       {budget_cvxpy:.4f} / {Q_BUDGET}")


    except Exception as e:
        print(f"\nAn error occurred: {e}")

Coupled Alpha-Fairness Solution Verification
Parameters: N=50, K=5, alpha=1.5, beta=2.5, Q=1000.0
--- Calculating solution via Closed-Form Formula ---
Calculation completed in 0.000182 seconds.

--- Solving problem with CVXPY and GUROBI ---
                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jun 23 01:48:22 PM: Your problem has 55 variables, 6 constraints, and 0 parameters.
(CVXPY) Jun 23 01:48:22 PM: It is compliant with the following grammars: 
(CVXPY) Jun 23 01:48:22 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 23 01:48:22 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 23 01:48:22 PM: Your problem is compiled with the CPP canonicalization backend.

An error occurred: Problem does not follow DCP rules. Specifically:
The followi

In [32]:
import numpy as np
import collections

def solve_closed_form(b, c, groups, Q, alpha, beta):
    """
    Calculates the optimal allocation d using the derived closed-form solution.
    This is the function we will be differentiating.
    """
    if beta == 1:
        raise ValueError("The closed-form solution is not defined for beta = 1.")
    
    n = len(b)
    d_star = np.zeros(n)
    gamma = beta + alpha - alpha * beta
    if gamma == 0:
        raise ValueError("gamma (alpha + beta - alpha*beta) cannot be zero.")

    S = {}
    H = {}
    Psi = {}
    
    for k, indices in groups.items():
        G_k = len(indices)
        b_k, c_k = b[indices], c[indices]
        
        S[k] = np.sum((c_k ** (-(1 - beta) / beta)) * (b_k ** ((1 - beta) / beta)))
        H[k] = np.sum((c_k ** ((beta - 1) / beta)) * (b_k ** ((1 - beta) / beta)))
        
        if S[k] == 0:
             raise ValueError(f"S_k for group {k} is zero. Cannot proceed.")

        Psi[k] = (
            (G_k ** ((alpha - 1) / gamma)) *
            (S[k] ** (-alpha / gamma)) *
            ((1 - beta) ** (alpha / gamma))
        )

    Xi = np.sum([H[k] * Psi[k] for k in groups])
    if Xi == 0:
        raise ValueError("Normalization constant Xi is zero. Cannot divide by zero.")

    for k, indices in groups.items():
        for i in indices:
            phi_i = (c[i] ** (-1 / beta)) * (b[i] ** ((1 - beta) / beta))
            d_star[i] = (Q / Xi) * Psi[k] * phi_i

    return d_star

def gradient_analytical(b, c, groups, Q, alpha, beta):
    """
    Computes the Jacobian matrix d(d*)/d(b) using the analytical formula.
    """
    print("--- Calculating Gradient via Analytical Formula ---")
    start_time = time.time()
    
    n = len(b)
    jacobian = np.zeros((n, n))

    # Pre-compute all terms from the forward pass (d_star calculation)
    gamma = beta + alpha - alpha * beta
    S, H, Psi = {}, {}, {}
    for k, indices in groups.items():
        G_k = len(indices)
        b_k, c_k = b[indices], c[indices]
        S[k] = np.sum((c_k ** (-(1 - beta) / beta)) * (b_k ** ((1 - beta) / beta)))
        H[k] = np.sum((c_k ** ((beta - 1) / beta)) * (b_k ** ((1 - beta) / beta)))
        Psi[k] = (
            (G_k ** ((alpha - 1) / gamma)) * (S[k] ** (-alpha / gamma)) * ((1 - beta) ** (alpha / gamma))
        )
    Xi = np.sum([H[k] * Psi[k] for k in groups])
    d_star = solve_closed_form(b, c, groups, Q, alpha, beta)
    
    # Map each individual j to their group m
    j_to_group_map = {j_idx: k for k, indices in groups.items() for j_idx in indices}

    # Calculate Jacobian column by column (for each b_j)
    for j in range(n):
        m = j_to_group_map[j] # Group of j
        
        # --- Derivatives of intermediate terms w.r.t. b_j ---
        
        # d(S_k)/d(b_j)
        dS_k_db_j = {k: 0 for k in groups}
        if S[m] != 0:
            dS_k_db_j[m] = ((1 - beta) / beta) * (c[j] ** (-(1-beta)/beta)) * (b[j] ** ((1-2*beta)/beta))
        
        # d(H_k)/d(b_j)
        dH_k_db_j = {k: 0 for k in groups}
        if H[m] != 0:
             dH_k_db_j[m] = ((1 - beta) / beta) * (c[j] ** ((beta-1)/beta)) * (b[j] ** ((1-2*beta)/beta))

        # d(Psi_k)/d(b_j)
        dPsi_k_db_j = {k: 0 for k in groups}
        if S[m] != 0:
            dPsi_k_db_j[m] = (-alpha / (gamma * S[m])) * Psi[m] * dS_k_db_j[m]
        
        # d(Xi)/d(b_j)
        dXi_db_j = dH_k_db_j[m] * Psi[m] + H[m] * dPsi_k_db_j[m]
        
        # --- Calculate full derivative d(d_i)/d(b_j) for all i ---
        for i in range(n):
            k = j_to_group_map[i] # Group of i

            # d(phi_i)/d(b_j)
            dphi_i_db_j = 0
            if i == j:
                dphi_i_db_j = ((1 - beta) / beta) * (c[i] ** (-1/beta)) * (b[i] ** ((1-2*beta)/beta))
            
            # d(Numerator)/d(b_j) where Numerator_i = Q * Psi_k * phi_i
            dN_i_db_j = Q * (dPsi_k_db_j[k] * ((c[i]**(-1/beta)) * (b[i]**((1-beta)/beta))) + Psi[k] * dphi_i_db_j)
            
            # Final assembly using quotient rule
            jacobian[i, j] = (dN_i_db_j * Xi - d_star[i] * Xi * dXi_db_j) / (Xi**2)
            # Simplified form
            jacobian[i, j] = (1/Xi) * dN_i_db_j - (d_star[i]/Xi) * dXi_db_j

    end_time = time.time()
    print(f"Calculation completed in {end_time - start_time:.6f} seconds.")
    return jacobian

def gradient_numerical(b, c, groups, Q, alpha, beta, epsilon=1e-7):
    """
    Computes the Jacobian matrix d(d*)/d(b) using finite differences.
    """
    print("\n--- Calculating Gradient via Finite Differences ---")
    start_time = time.time()
    n = len(b)
    jacobian = np.zeros((n, n))
    
    for j in range(n):
        b_plus = b.copy()
        b_minus = b.copy()
        
        b_plus[j] += epsilon
        b_minus[j] -= epsilon
        
        d_plus = solve_closed_form(b_plus, c, groups, Q, alpha, beta)
        d_minus = solve_closed_form(b_minus, c, groups, Q, alpha, beta)
        
        jacobian[:, j] = (d_plus - d_minus) / (2 * epsilon)

    end_time = time.time()
    # Note: This is slow because it calls the full solver 2*n times.
    print(f"Calculation completed in {end_time - start_time:.6f} seconds.")
    return jacobian

if __name__ == '__main__':
    # --- 1. Define Problem Parameters ---
    N_INDIVIDUALS = 20
    N_GROUPS = 4
    ALPHA = 0.5
    BETA = 0.7
    Q_BUDGET = 100.0

    print("=" * 60)
    print("Gradient Verification for Closed-Form Solution")
    print(f"Parameters: N={N_INDIVIDUALS}, K={N_GROUPS}, alpha={ALPHA}, beta={BETA}, Q={Q_BUDGET}")
    print("=" * 60)

    # --- 2. Generate Random Problem Data ---
    np.random.seed(0)
    b_coeffs = np.random.uniform(1, 10, size=N_INDIVIDUALS)
    c_coeffs = np.random.uniform(0.5, 5, size=N_INDIVIDUALS)
    
    group_assignments = np.random.randint(0, N_GROUPS, size=N_INDIVIDUALS)
    groups_dict = collections.defaultdict(list)
    for i, k in enumerate(group_assignments):
        groups_dict[k].append(i)
    # Ensure all groups are present if any were missed by random assignment
    active_groups = sorted(groups_dict.keys())
    for k in range(N_GROUPS):
        if k not in active_groups:
            # Move an individual to the empty group
            move_from_group = active_groups[k % len(active_groups)]
            if len(groups_dict[move_from_group]) > 1:
                moved_indiv = groups_dict[move_from_group].pop()
                groups_dict[k].append(moved_indiv)

    # --- 3. Run Both Methods ---
    try:
        grad_analytic = gradient_analytical(b_coeffs, c_coeffs, groups_dict, Q_BUDGET, ALPHA, BETA)
        grad_numeric = gradient_numerical(b_coeffs, c_coeffs, groups_dict, Q_BUDGET, ALPHA, BETA)

        # --- 4. Compare the Results ---
        print("\n" + "=" * 60)
        print("--- Comparison of Gradient Results ---")
        
        # Show a slice of the Jacobian matrices
        print("\nAnalytical Gradient (top-left 5x5 slice):\n", grad_analytic[:5, :5])
        print("\nNumerical Gradient (top-left 5x5 slice):\n", grad_numeric[:5, :5])

        # Calculate the difference using the Frobenius norm
        difference_norm = np.linalg.norm(grad_analytic - grad_numeric)
        print(f"\nFrobenius norm of the difference between the two Jacobians: {difference_norm:.8f}")

        if np.allclose(grad_analytic, grad_numeric, rtol=1e-3):
            print("\n✅ SUCCESS: The analytical gradient matches the numerical gradient.")
        else:
            print("\n❌ FAILURE: The gradients do not match.")

    except Exception as e:
        print(f"\nAn error occurred: {e}")

Gradient Verification for Closed-Form Solution
Parameters: N=20, K=4, alpha=0.5, beta=0.7, Q=100.0
--- Calculating Gradient via Analytical Formula ---
Calculation completed in 0.001083 seconds.

--- Calculating Gradient via Finite Differences ---
Calculation completed in 0.003945 seconds.

--- Comparison of Gradient Results ---

Analytical Gradient (top-left 5x5 slice):
 [[ 0.05441176 -0.00060824 -0.00244581 -0.00103716 -0.00208567]
 [-0.00091173  0.03844871 -0.00207572 -0.00088022 -0.00177007]
 [-0.0050353  -0.00285087  0.19427406 -0.00486125 -0.00977576]
 [-0.00126002 -0.0007134  -0.00286867  0.06517271 -0.01595772]
 [-0.00802913 -0.00454591 -0.01827977 -0.05056593  0.46978167]]

Numerical Gradient (top-left 5x5 slice):
 [[ 0.05441176 -0.00060824 -0.00244581 -0.00103716 -0.00208567]
 [-0.00091173  0.03844871 -0.00207572 -0.00088022 -0.00177007]
 [-0.0050353  -0.00285088  0.19427405 -0.00486126 -0.00977576]
 [-0.00126002 -0.0007134  -0.00286867  0.06517271 -0.01595772]
 [-0.00802913 -

In [29]:
import numpy as np
import collections

def solve_closed_form(b, c, group_idx, Q, alpha, beta):
    """
    (Helper function from previous answer)
    Calculates the optimal allocation d using the closed-form solution.
    """
    if beta == 1 or alpha in [1, 0, float('inf')]:
        # The general closed-form solution is not defined for these special cases.
        # This helper is only used to generate a plausible 'd' for testing.
        # In a real scenario, a different solver would be needed for these alphas.
        print(f"Warning: Using generalized formula for alpha={alpha}, beta={beta} to get a 'd' vector. This is for demonstration only.")

    n = len(b)
    d_star = np.zeros(n)
    gamma = beta + alpha - alpha * beta
    if gamma == 0:
        # Avoid division by zero if, e.g., alpha=2, beta=2
        gamma = 1e-12

    unique_groups = np.unique(group_idx)
    S, H, Psi = {}, {}, {}

    for k in unique_groups:
        members_mask = (group_idx == k)
        G_k = np.sum(members_mask)
        b_k, c_k = b[members_mask], c[members_mask]

        S[k] = np.sum((c_k ** (-(1 - beta) / beta)) * (b_k ** ((1 - beta) / beta)))
        H[k] = np.sum((c_k ** ((beta - 1) / beta)) * (b_k ** ((1 - beta) / beta)))
        
        if S[k] == 0: S[k] = 1e-12
        Psi[k] = ((G_k ** ((alpha - 1) / gamma)) * (S[k] ** (-alpha / gamma)) * ((1 - beta) ** (alpha / gamma)))

    Xi = np.sum([H[k] * Psi[k] for k in unique_groups])
    if Xi == 0: Xi = 1e-12

    for k in unique_groups:
        members_mask = (group_idx == k)
        phi = (c[members_mask] ** (-1 / beta)) * (b[members_mask] ** ((1 - beta) / beta))
        d_star[members_mask] = (Q / Xi) * Psi[k] * phi

    return d_star


def calculate_objective_value(d, b, group_idx, alpha, beta):
    """
    Calculates the objective value for the coupled alpha-fairness problem.

    This function correctly handles the special cases for alpha and beta.

    Args:
        d (np.ndarray): The allocation vector d.
        b (np.ndarray): The vector of benefit coefficients.
        group_idx (np.ndarray): An array of group assignments for each individual.
        alpha (float or str): The outer fairness parameter. Can be 'inf'.
        beta (float): The inner fairness parameter.

    Returns:
        float: The final scalar objective value.
    """
    # --- Step 1: Calculate all group utilities (mu_k) ---
    
    # Calculate the argument of the inner utility function, y_i = b_i * d_i
    # Add a small epsilon for numerical stability with log operations
    y = b * d + 1e-12

    # Calculate individual utilities g_beta(y_i)
    if abs(beta - 1.0) < 1e-9:
        # Case beta = 1 (logarithmic utility)
        g_beta_values = np.log(y)
    else:
        # General case for beta
        g_beta_values = (y**(1 - beta)) / (1 - beta)

    # Aggregate to find the mean utility for each group
    unique_groups = np.unique(group_idx)
    mu_k_values = np.zeros(len(unique_groups))
    for i, k in enumerate(unique_groups):
        members_mask = (group_idx == k)
        mu_k_values[i] = np.mean(g_beta_values[members_mask])

    # --- Step 2: Apply the outer fairness function (f_alpha) and aggregate ---

    if alpha == float('inf') or str(alpha).lower() == 'inf':
        # Case alpha = inf (Max-Min Fairness)
        # The value of the Rawlsian objective is the utility of the worst-off group.
        objective_value = np.min(mu_k_values)

    elif abs(alpha - 1.0) < 1e-9:
        # Case alpha = 1 (Proportional Fairness / Logarithmic Utility)
        # Objective is sum(log(mu_k))
        objective_value = np.sum(np.log(mu_k_values + 1e-12))
    
    elif abs(alpha - 0.0) < 1e-9:
        # Case alpha = 0 (Utilitarian)
        # Objective is sum(mu_k)
        objective_value = np.sum(mu_k_values)
        
    else:
        # General case for alpha
        # Objective is sum(mu_k^(1-alpha) / (1-alpha))
        f_alpha_values = (mu_k_values**(1 - alpha)) / (1 - alpha)
        objective_value = np.sum(f_alpha_values)
        
    return objective_value

if __name__ == '__main__':
    # --- 1. Define Problem Parameters ---
    N_INDIVIDUALS = 5000
    N_GROUPS = 5
    BETA = 2  # The inner fairness parameter
    Q_BUDGET = 1000.0

    print("=" * 60)
    print("Calculating Objective Value for Different Alpha Values")
    print(f"Parameters: N={N_INDIVIDUALS}, K={N_GROUPS}, beta={BETA}, Q={Q_BUDGET}")
    print("=" * 60)

    # --- 2. Generate Random Problem Data ---
    np.random.seed(42)
    b_coeffs = np.random.uniform(1, 10, size=N_INDIVIDUALS)
    c_coeffs = np.random.uniform(0.5, 5, size=N_INDIVIDUALS)
    group_idx_array = np.random.randint(0, N_GROUPS, size=N_INDIVIDUALS)

    # --- 3. Test the function for various alpha values ---
    
    # We generate a single 'd' vector using a representative alpha (e.g., 0.5)
    # and then evaluate how different fairness objectives would score this *same* allocation.
    print("Generating a sample allocation vector d* using alpha=0.5...")
    d_optimal_sample = solve_closed_form(b_coeffs, c_coeffs, group_idx_array, Q_BUDGET, alpha=0.5, beta=BETA)
    print("Done.\n")
    
    alphas_to_test = [
        0,          # Utilitarian
        0.5,        # General case (concave)
        1,          # Proportional / Logarithmic
        2.0,        # General case (more inequality averse)
        'inf'       # Max-Min / Rawlsian
    ]

    for alpha_test in alphas_to_test:
        obj_val = calculate_objective_value(
            d=d_optimal_sample,
            b=b_coeffs,
            group_idx=group_idx_array,
            alpha=alpha_test,
            beta=BETA
        )
        description = {
            0: "Utilitarian (sum of group utilities)",
            1: "Proportional (sum of log group utilities)",
            'inf': "Max-Min (minimum group utility)"
        }.get(alpha_test, f"General case alpha={alpha_test}")
        
        print(f"Objective Value for alpha = {str(alpha_test):<4} ({description}): {obj_val:.6f}")

Calculating Objective Value for Different Alpha Values
Parameters: N=5000, K=5, beta=2, Q=1000.0
Generating a sample allocation vector d* using alpha=0.5...
Done.

Objective Value for alpha = 0    (Utilitarian (sum of group utilities)): -14.785312
Objective Value for alpha = 0.5  (General case alpha=0.5): nan
Objective Value for alpha = 1    (Proportional (sum of log group utilities)): nan
Objective Value for alpha = 2.0  (General case alpha=2.0): 1.691397
Objective Value for alpha = inf  (Max-Min (minimum group utility)): -3.021280


/var/folders/zm/yg4qm2yj0vs5j9t4_s31v5mw0000gn/T/ipykernel_39522/2099695817.py:42: ComplexWarning: Casting complex values to real discards the imaginary part
  d_star[members_mask] = (Q / Xi) * Psi[k] * phi
/var/folders/zm/yg4qm2yj0vs5j9t4_s31v5mw0000gn/T/ipykernel_39522/2099695817.py:104: RuntimeWarning: invalid value encountered in sqrt
  f_alpha_values = (mu_k_values**(1 - alpha)) / (1 - alpha)
/var/folders/zm/yg4qm2yj0vs5j9t4_s31v5mw0000gn/T/ipykernel_39522/2099695817.py:94: RuntimeWarning: invalid value encountered in log
  objective_value = np.sum(np.log(mu_k_values + 1e-12))


In [30]:
print()